## Old

### Importieren der Klassen

In [ ]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import accelerate

### Festlegen des Modells

In [ ]:
model_id="/home/thsch026/masterarbeit/models/generated/llm-awq/Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"

### Initialisieren des Modells

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=quantization_config, #load_in_8bit=True,#
    device_map='auto',
    token='<EOL>'
)
#peft_model = "BojanaBas/Meta-Llama-3-8B-Instruct-pqa-10"
#model = PeftModel.from_pretrained(base_model, peft_model)

### Inferenz Test

### Autoawq library is needed to test inference with AWQ Models

In [ ]:
pip install autoawq

In [ ]:
# Eingabetext
input_text = "Once upon a time"
#input_text = "Why should I eat healthy?"

# Tokenisiere den Eingabetext
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Messe die Zeit vor der Inferenz
start_time = time.time()

# Führe die Inferenz durch
outputs = model.generate(input_ids, max_length=50)

# Messe die Zeit nach der Inferenz
end_time = time.time()

# Berechne die Inferenz-Zeit
inference_time = end_time - start_time

# Dekodiere die Ausgabe
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
number_of_words = len(output_text.split())
number_of_tokens = len(outputs[0])

token_per_second = number_of_tokens/inference_time

print(f"Inferenz-Zeit: {inference_time} Sekunden")
print(f"Output: {output_text}")
print("Tokens per second: ", token_per_second)



In [ ]:
import csv

# Die neuen Daten, die hinzugefügt werden sollen
neue_daten = ['Lukas', 28]

# Öffne die Datei im Anhangsmodus ('a' steht für append)
with open('daten.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    
    # Schreibe die neuen Daten in die CSV-Datei
    writer.writerow(neue_daten)

print("Neue Daten erfolgreich hinzugefügt.")

In [ ]:
pip install sentencepiece

## Evaluierung Modelle

### Sentencepiece muss ggf. noch mal installiert werden für Mistral (fast tokenizer)

In [ ]:
pip install sentencepiece

### Laden der libaries und festlegen der GPU

In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!echo $CUDA_VISIBLE_DEVICES
# torch.cuda.set_device(1)

0


In [2]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import accelerate
import csv

### Ergebnis Dateien

In [3]:
full_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference_full.csv"
data_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference_data_only.csv"

##  1. Referenz Modelle

### Llama-3-8B-Instruct

In [ ]:
model_id ="Meta-Llama-3-8B-Instruct"
model_loc = "meta-llama/Meta-Llama-3-8B-Instruct"

### Mistral-7B-Instruct-v0.2

In [ ]:
model_id = "Mistral-7B-Instruct-v0.2"
model_loc = "mistralai/Mistral-7B-Instruct-v0.2"

### Llama-2-7b-chat-hf

In [ ]:
model_id = "Llama-2-7b-chat-hf"
model_loc = "meta-llama/Llama-2-7b-chat-hf"

## 2. AWQ Modelle

### Llama-3-8B-Instruct-HF-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"
model_loc = model_path + model_id

### Mistral-7B-Instruct-v0.2-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Mistral-7B-Instruct-v0.2-AWQ-4bit"
model_loc = model_path + model_id

### Llama-2-7b-chat-HF-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Llama-2-7b-chat-HF-AWQ-4bit"
model_loc = model_path + model_id

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/awq/quant/"
model_id = "Meta-Llama-3-8B-Instruct-HF-w8-g128-awq-v2.pt"
model_loc = model_path + model_id

## 3. Pruned Models (shortened-LLM)

### Meta-Llama-3-8B

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Meta-Llama-3-8B"
model_id = "Meta-Llama-3-8B_pruned_llm-shortened"

### Llama-2-7b-hf (schlechtes Ergebnis)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Llama-2-7b-hf"
model_id = "Llama-2-7b-hf_pruned_llm-shortened"

### Mistral-7B-Instruct

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Mistral-7B-Instruct"
model_id = "Mistral-7B-Instruct_pruned_llm-shortened"

## 4. Lora Models

#### Meta-Llama-3-8B_ms-marco (Kein gutes Ergebnis)

In [12]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/Meta-Llama-3-8B_ms-marco/model"
model_id = "Meta-Llama-3-8B_ms-marco"

##  5. Qlora Models

### Meta-Llama-3-8B-Instruct_qlora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Meta-Llama-3-8B-Instruct_qlora"
model_id = "Meta-Llama-3-8B-Instruct_qlora"

### Mistral-7B-Instruct-v0.2_qlora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Mistral-7B-Instruct-v0.2_qlora"
model_id = "Mistral-7B-Instruct-v0.2_qlora"

### llama-2-7b-chat_qlora (Fehlt noch)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/llama-2-7b-chat_qlora"
model_id = "llama-2-7b-chat_qlora"

## 6. Prune + Lora Models (shortened-LLM)

#### Meta-Llama-3-8B

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Meta-Llama-3-8B_prune_lora"
model_id = "Meta-Llama-3-8B_prune_lora"

#### Llama-2-7b-hf_prune_lora

In [8]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Llama-2-7b-hf_prune_lora"
model_id = "Llama-2-7b-hf_prune_lora"

#### Mistral-7B-Instruct_prune_lora

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Mistral-7B-Instruct_prune_lora"
model_id = "Mistral-7B-Instruct_prune_lora"

### Prune + Lora + AWQ Models

#### Mistral-7B-Instruct_prune_lora_awq

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Mistral-7B-Instruct-v02_prune_lora_awq"
model_id = "Mistral-7B-Instruct_prune_lora_awq"

## Inference Task

### Laden des Modells

In [5]:

tokenizer = AutoTokenizer.from_pretrained(model_loc)

#quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
#quantization_config = BitsAndBytesConfig(bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_loc,
    trust_remote_code=True,
    # Bei AWQ Modellen keine quantization config angeben
   # quantization_config=quantization_config, #load_in_8bit=True
    device_map='cuda'
)


### Interferenz mit Ergebnis Speicherung

In [6]:
# Funktion für das Speichern der Werte

def append_data(result):
    
    with open(full_csv, mode='a', newline='') as file:
        full_writer = csv.writer(file)
        full_writer.writerow(result)
        
    with open(data_csv, mode='a', newline='') as file:
        data_writer = csv.writer(file)
        data_writer.writerow(result[0:4])

                
device = 'cuda'

question_list = ["Once upon a time", "Why should I eat healthy?", "Is there a second life after death?", "What is the most common injury?","Who is Konrad Zuse?"]

for question in question_list:
    input_ids = tokenizer.encode(question, return_tensors='pt')
    # Änderung dr Zeile weil es Probleme mit der Ausführung gab (cuda/cpu anomalie)
    input_ids = tokenizer(question, return_tensors="pt").input_ids.to(device)
    input_ids.to('cuda:0')
    start_time = time.time()

    # Führe die Inferenz durch
    outputs = model.generate(input_ids, max_length=200)

    # Messe die Zeit nach der Inferenz
    end_time = time.time()

    # Berechne die Inferenz-Zeit
    inference_time = end_time - start_time

    # Dekodiere die Ausgabe
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Berchnung des Ergebnis
    #number_of_words = len(output_text.split())
    number_of_tokens = len(outputs[0])
    memory_usage = round((torch.cuda.memory_allocated() / (1024 **3)),2)
    token_per_second = number_of_tokens/inference_time
    result = [model_id,token_per_second, inference_time, memory_usage, question, output_text]
    print(result)
    append_data(result)
              
    
                                                                
    

    
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Mistral-7B-Instruct_prune_lora_awq', 2.673348976815264, 74.81252980232239, 3.65, 'Once upon a time', "Once upon a time, people were luddites and computers were uncommon. Then, one night while sitting up for a pony ride in Fort Bramshutte's grandstand, something was drawn in my notebook. A beast is drawn. I made it mine. I had a game, called 'The Beast' (a version of which is still available on the app store) and began to obsessively research, inventing and implementing a world-replacement engine in my game, and creating a vast world that grew from a few simple rules.\n\nThe game was called Quest, and it was like the computer-driven worlds that eventually arrived and made my dreams of the endless possibilities of the imaginary world. My friend Bill and I created this, and we were called the Beast Corporation, and it seemed that our game had become the ultimate expression of our art.\n\nBut, like nature, our work went on, and the years passed as we"]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Mistral-7B-Instruct_prune_lora_awq', 3.097978716356849, 43.57680034637451, 3.65, 'Why should I eat healthy?', 'Why should I eat healthy?\n\nYou should eat healthy to improve your physical and mental health, boost your immune system, maintain your weight, and prevent chronic diseases such as obesity, type 2 diabetes, heart disease, and certain types of cancer.\n\nWhat makes food healthy?\n\nTo be considered healthy, food often contains nutrients such as vitamins, minerals, fiber, and antioxidants that are essential for the growth, development, and functioning of the body. It should also include a variety of types of foods, including fruits, vegetables, whole grains, lean protein, and healthy fats.']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Mistral-7B-Instruct_prune_lora_awq', 3.0871269332784608, 64.78515601158142, 3.65, 'Is there a second life after death?', "Is there a second life after death? Are humans the only intelligent beings on Earth? The debate on these topics has been going on for centuries.\n\nIf you have a specific question, or topic you'd like our scientists to explore, send our team a message – or fill out this form:\n\nSen. John Cornyn, Texas Republican, called Democrats’ move to move to a vote on banning semi-automatic guns an ‘unraveling of American constitutional liberties.’ CNN anchor Brian Bremmel suggested that his statement would be “mostly out of line with the typical views of his fellow [senators], that is, Democrats would interpret such a measure as reasonable, perhaps even a requirement.\n\nIn contrast to the senator’s words of the ‘unraveling of American constitutional liberties,' CNN correspondent Elizabeth Cohn described her country’s right to bear ammunition as “savage.''\n\n"]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Mistral-7B-Instruct_prune_lora_awq', 3.071416592066884, 65.11653304100037, 3.65, 'What is the most common injury?', 'What is the most common injury?\n\nImpact with the hard ground or track is the number one cause of injury. Another common injury is hypochondriosis.\n\nCan an injury be prevented?\n\nYes. Many injuries can be prevented by implementing certain risk management strategies. This can include proper conditioning, strength and flexibility training, equipment properly fitted, and adherence to speed and running rules.\n\nWhy can\'t athletes train on Astrolite tiles?\n\nAstrolite tiles are not the best surface for sportscerbony training because the surface is not the correct ratio of water, oxygen, and food for the athletes to perform their best. Athletes trained on Astrolite also tend to get sick easily, and not even the sickest of athletes would be seen training on this surface.\n\nWhat is a bumps game?\n\nIt\'s actually called "Bumpsies," it\'s when two sides']
['Mistral-7B-I

In [7]:
del model # Specify variable
torch.cuda.empty_cache()